In [3]:
import pandas as pd
import ccxt

## Data obtaining from ccxt library 
--------------------------------------------
motivation, get high frequency data from exchange
exchange(s) should be decided \
Another thing is that cryptocurrency against which currency should be decided. \
anlamadığım bir durum var doları cryptocurrency -lere karşı bulamıyorum

In [ ]:
#https://backtest-rookies.com/2018/03/08/download-cryptocurrency-data-with-ccxt/
#https://www.linkedin.com/pulse/download-historical-data-all-cryptocoins-ccxt-gursel-karacor/?articleId=6636391574811090944   
## final: https://github.com/codeninja/CCXT-Historical-Data/blob/master/Binance%20Historical%20Data%20.ipynb

In [4]:
import os
from pathlib import Path

import sys
import csv

import ccxt
# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if len(ohlcv) == 0:
            break
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("../data/0_raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles+
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

In [ ]:
#BUSD ile çok az veri çekilebildiği için usdt yi de çektim.

In [5]:
scrape_candles_to_csv('./btc_usdt_6h.csv', 'binance', 3, 'BTC/USDT', '6h', '2017-01-0100:00:00Z', 1000)

1000 BTC/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
2000 BTC/USDT candles in total from 2020-03-31T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3000 BTC/USDT candles in total from 2019-07-25T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3999 BTC/USDT candles in total from 2018-11-17T06:00:00.000Z to 2021-08-13T00:00:00.000Z
4998 BTC/USDT candles in total from 2018-03-12T06:00:00.000Z to 2021-08-13T00:00:00.000Z
5823 BTC/USDT candles in total from 2017-08-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 5823 candles from 2017-08-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./btc_usdt_6h.csv


In [6]:
scrape_candles_to_csv('./eth_usdt_6h.csv', 'binance', 3, 'ETH/USDT', '6h', '2017-01-0100:00:00Z', 1000)

1000 ETH/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
2000 ETH/USDT candles in total from 2020-03-31T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3000 ETH/USDT candles in total from 2019-07-25T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3999 ETH/USDT candles in total from 2018-11-17T06:00:00.000Z to 2021-08-13T00:00:00.000Z
4998 ETH/USDT candles in total from 2018-03-12T06:00:00.000Z to 2021-08-13T00:00:00.000Z
5823 ETH/USDT candles in total from 2017-08-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 5823 candles from 2017-08-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./eth_usdt_6h.csv


In [7]:
scrape_candles_to_csv('./ltc_usdt_6h.csv', 'binance', 3, 'LTC/USDT', '6h', '2017-01-0100:00:00Z', 1000)

1000 LTC/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
2000 LTC/USDT candles in total from 2020-03-31T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3000 LTC/USDT candles in total from 2019-07-25T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3999 LTC/USDT candles in total from 2018-11-17T06:00:00.000Z to 2021-08-13T00:00:00.000Z
4998 LTC/USDT candles in total from 2018-03-12T06:00:00.000Z to 2021-08-13T00:00:00.000Z
5351 LTC/USDT candles in total from 2017-12-13T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 5351 candles from 2017-12-13T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./ltc_usdt_6h.csv


In [8]:
for filename in ["btc_usdt_6h.csv", "eth_usdt_6h.csv", "ltc_usdt_6h.csv"]:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index,unit='s')
    df.to_csv("../data/0_raw/" + filename)

In [ ]:
#https://coinmarketcap.com/all/views/all/

In [11]:
scrape_candles_to_csv('./ada_usdt_6h.csv', 'binance', 3, 'ADA/USDT', '6h', '2017-01-0100:00:00Z', 1000)

1000 ADA/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
2000 ADA/USDT candles in total from 2020-03-31T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3000 ADA/USDT candles in total from 2019-07-25T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3999 ADA/USDT candles in total from 2018-11-17T06:00:00.000Z to 2021-08-13T00:00:00.000Z
4855 ADA/USDT candles in total from 2018-04-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 4855 candles from 2018-04-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./ada_usdt_6h.csv


In [12]:
scrape_candles_to_csv('./xrp_usdt_6h.csv', 'binance', 3, 'XRP/USDT', '6h', '2017-01-0100:00:00Z', 1000)

1000 XRP/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
2000 XRP/USDT candles in total from 2020-03-31T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3000 XRP/USDT candles in total from 2019-07-25T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3999 XRP/USDT candles in total from 2018-11-17T06:00:00.000Z to 2021-08-13T00:00:00.000Z
4786 XRP/USDT candles in total from 2018-05-04T06:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 4786 candles from 2018-05-04T06:00:00.000Z to 2021-08-13T00:00:00.000Z to ./xrp_usdt_6h.csv


In [13]:
scrape_candles_to_csv('./doge_usdt_6h.csv', 'binance', 3, 'DOGE/USDT', '6h', '2017-01-0100:00:00Z', 1000)

1000 DOGE/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
2000 DOGE/USDT candles in total from 2020-03-31T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3000 DOGE/USDT candles in total from 2019-07-25T06:00:00.000Z to 2021-08-13T00:00:00.000Z
3079 DOGE/USDT candles in total from 2019-07-05T12:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 3079 candles from 2019-07-05T12:00:00.000Z to 2021-08-13T00:00:00.000Z to ./doge_usdt_6h.csv


In [14]:
scrape_candles_to_csv('./dot_usdt_6h.csv', 'binance', 3, 'DOT/USDT', '6h', '2017-01-0100:00:00Z', 1000) 
#bu azmış çıkarabiliriz

1000 DOT/USDT candles in total from 2020-12-06T06:00:00.000Z to 2021-08-13T00:00:00.000Z
1438 DOT/USDT candles in total from 2020-08-18T18:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 1438 candles from 2020-08-18T18:00:00.000Z to 2021-08-13T00:00:00.000Z to ./dot_usdt_6h.csv


In [15]:
for filename in ['./ada_usdt_6h.csv', './xrp_usdt_6h.csv', './dot_usdt_6h.csv']:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index, unit='s')
    df.to_csv("../data/0_raw/" + filename)

In [16]:
scrape_candles_to_csv('./ada_usdt_1d.csv', 'binance', 3, 'ADA/USDT', '1d', '2017-01-0100:00:00Z', 1000)
scrape_candles_to_csv('./xrp_usdt_1d.csv', 'binance', 3, 'XRP/USDT', '1d', '2017-01-0100:00:00Z', 1000)
scrape_candles_to_csv('./dot_usdt_1d.csv', 'binance', 3, 'DOT/USDT', '1d', '2017-01-0100:00:00Z', 1000) 
#bu azmış çıkarabiliriz

1000 ADA/USDT candles in total from 2018-11-18T00:00:00.000Z to 2021-08-13T00:00:00.000Z
1215 ADA/USDT candles in total from 2018-04-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 1215 candles from 2018-04-17T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./ada_usdt_1d.csv
1000 XRP/USDT candles in total from 2018-11-18T00:00:00.000Z to 2021-08-13T00:00:00.000Z
1198 XRP/USDT candles in total from 2018-05-04T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 1198 candles from 2018-05-04T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./xrp_usdt_1d.csv
361 DOT/USDT candles in total from 2020-08-18T00:00:00.000Z to 2021-08-13T00:00:00.000Z
Saved 361 candles from 2020-08-18T00:00:00.000Z to 2021-08-13T00:00:00.000Z to ./dot_usdt_1d.csv


In [ ]:
for filename in ['./ada_usdt_1d.csv', './xrp_usdt_1d.csv', './dot_usdt_1d.csv']:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index, unit='s')
    df.to_csv("../data/0_raw/" + filename)